Importing data from file

#importing data
import pandas as p

In [ ]:
import re
# read file
text = open('dataset-1.txt', 'r', encoding='utf-8').read()
text = text.replace('\r\n', '\n').replace('\r', '\n')
# text = re.sub(r'(\n\s*\n)+', ' . . . . . ', text) # replace multiple newlines with ' . . . . . '
text = re.sub(r'[-]+', ' ', text) # replace hyphens with space
text = re.sub(r'[^a-zA-Z0-9 \.\n]', '', text) # remove special characters except period
text = re.sub(r'\n\n', ' <EOS> ', text) # replace newlines with space
text = re.sub(r'\.+', '.', text).strip() # collapse multiple spaces into one
text = re.sub(r'\.', ' . ', text) # surround periods with spaces
text = re.sub(r'\s+', ' ', text).strip() # collapse multiple spaces into one
text = text.lower() # convert to lowercase
text = text[:len(text)]  # use only last 90% of the text

In [2]:
window_size = 4

paragraphs = text.split(' <eos> ')
# words = [word for para in paragraphs for word in para.split() + ['<EOS>']]
words = []
for para in paragraphs:
    words.extend(para.split(' ') + ['<PAD>'] * window_size)

In [3]:
words
# paragraphs

['well',
 'prince',
 'so',
 'genoa',
 'and',
 'lucca',
 'are',
 'now',
 'just',
 'family',
 'estates',
 'of',
 'the',
 'buonapartes',
 '.',
 'but',
 'i',
 'warn',
 'you',
 'if',
 'you',
 'dont',
 'tell',
 'me',
 'that',
 'this',
 'means',
 'war',
 'if',
 'you',
 'still',
 'try',
 'to',
 'defend',
 'the',
 'infamies',
 'and',
 'horrors',
 'perpetrated',
 'by',
 'that',
 'antichrist',
 'i',
 'really',
 'believe',
 'he',
 'is',
 'antichrist',
 'i',
 'will',
 'have',
 'nothing',
 'more',
 'to',
 'do',
 'with',
 'you',
 'and',
 'you',
 'are',
 'no',
 'longer',
 'my',
 'friend',
 'no',
 'longer',
 'my',
 'faithful',
 'slave',
 'as',
 'you',
 'call',
 'yourself',
 'but',
 'how',
 'do',
 'you',
 'do',
 'i',
 'see',
 'i',
 'have',
 'frightened',
 'you',
 'sit',
 'down',
 'and',
 'tell',
 'me',
 'all',
 'the',
 'news',
 '.',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 'it',
 'was',
 'in',
 'july',
 '1805',
 'and',
 'the',
 'speaker',
 'was',
 'the',
 'well',
 'known',
 'anna',
 'pavlovna',
 'schere

In [4]:
window_size = 4

X, Y = [], []
for i in range(len(words)):
    start_idx = max(0, i - window_size)
    context = words[start_idx:i]
    context = ['<PAD>'] * (window_size - len(context)) + context  # left padding
    target = words[i]
    
    X.append(context)
    Y.append(target)


for i in range(100):
    print(' '.join(X[i]), '--->', Y[i])


<PAD> <PAD> <PAD> <PAD> ---> well
<PAD> <PAD> <PAD> well ---> prince
<PAD> <PAD> well prince ---> so
<PAD> well prince so ---> genoa
well prince so genoa ---> and
prince so genoa and ---> lucca
so genoa and lucca ---> are
genoa and lucca are ---> now
and lucca are now ---> just
lucca are now just ---> family
are now just family ---> estates
now just family estates ---> of
just family estates of ---> the
family estates of the ---> buonapartes
estates of the buonapartes ---> .
of the buonapartes . ---> but
the buonapartes . but ---> i
buonapartes . but i ---> warn
. but i warn ---> you
but i warn you ---> if
i warn you if ---> you
warn you if you ---> dont
you if you dont ---> tell
if you dont tell ---> me
you dont tell me ---> that
dont tell me that ---> this
tell me that this ---> means
me that this means ---> war
that this means war ---> if
this means war if ---> you
means war if you ---> still
war if you still ---> try
if you still try ---> to
you still try to ---> defend
still try t

In [5]:
from collections import Counter

word_counts = Counter(words)
vocab = sorted(word_counts.keys()) 
vocab_size = len(vocab)

print("Vocabulary Size:", vocab_size)

Vocabulary Size: 13078


In [6]:
most_common_10 = word_counts.most_common(10)
least_common_10 = word_counts.most_common()[-10:]

print("10 Most Frequent Words:", most_common_10)
print("10 Least Frequent Words:", least_common_10)


10 Most Frequent Words: [('<PAD>', 24064), ('the', 16098), ('.', 14072), ('and', 11144), ('to', 8284), ('of', 6499), ('a', 5472), ('he', 5084), ('in', 4464), ('his', 4189)]
10 Least Frequent Words: [('enterprise', 1), ('balashevs', 1), ('tethered', 1), ('trappings', 1), ('curling', 1), ('equestrian', 1), ('bracelets', 1), ('necklaces', 1), ('theatrically', 1), ('julner', 1)]


In [7]:
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using Apple Silicon GPU (MPS)")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using NVIDIA GPU (CUDA)")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using Apple Silicon GPU (MPS)


In [8]:
stoi = {word: i for i, word in enumerate(vocab)}
itos = {i: word for word, i in stoi.items()}
emb_dim = 64
emb = torch.nn.Embedding(len(stoi), emb_dim)
print(stoi)
print(itos)

{'.': 0, '1': 1, '10': 2, '10000': 3, '100000': 4, '11': 5, '13th': 6, '15': 7, '15000': 8, '150000': 9, '160000': 10, '178': 11, '17th': 12, '18': 13, '1805': 14, '1806': 15, '1807': 16, '1808': 17, '1809': 18, '1810': 19, '1811': 20, '1812': 21, '18th': 22, '2': 23, '217': 24, '23': 25, '23rd': 26, '24th': 27, '25th': 28, '26': 29, '27': 30, '27th': 31, '3': 32, '30': 33, '30000': 34, '3rd': 35, '4': 36, '4th': 37, '5': 38, '6': 39, '7': 40, '700': 41, '70000': 42, '7th': 43, '800': 44, '80000': 45, '9th': 46, '<PAD>': 47, '<eos>': 48, 'a': 49, 'aah': 50, 'ab': 51, 'aback': 52, 'abacus': 53, 'abandon': 54, 'abandoned': 55, 'abandoning': 56, 'abasement': 57, 'abashed': 58, 'abbe': 59, 'abbes': 60, 'abbreviations': 61, 'abc': 62, 'abduction': 63, 'abductors': 64, 'ability': 65, 'able': 66, 'abnegation': 67, 'abnormally': 68, 'abolishing': 69, 'abominable': 70, 'about': 71, 'above': 72, 'aboveboard': 73, 'abreast': 74, 'abroad': 75, 'abrupt': 76, 'abruptly': 77, 'absence': 78, 'absent':

In [9]:
print(emb.weight.shape)  # should be (vocab_size, emb_dim)

torch.Size([13078, 64])


In [10]:
X_encoded = [[stoi[word] for word in seq] for seq in X]
Y_encoded = [stoi[word] for word in Y] 

In [11]:
print(X_encoded[:5])

[[47, 47, 47, 47], [47, 47, 47, 12733], [47, 47, 12733, 8762], [47, 12733, 8762, 10638], [12733, 8762, 10638, 4893]]


In [12]:
stoi['<PAD>']

47

In [13]:
X = torch.tensor(X_encoded, dtype=torch.long).to(device)
Y = torch.tensor(Y_encoded, dtype=torch.long).to(device)

In [14]:
emb.weight

Parameter containing:
tensor([[-0.8350,  1.6432, -1.8157,  ...,  0.4247,  0.7649,  1.3068],
        [ 1.2198,  1.2959, -0.1070,  ...,  1.5618,  0.4356, -1.5392],
        [-2.9743,  1.2427,  0.3338,  ...,  0.3693, -0.2189, -0.7957],
        ...,
        [ 0.3762,  1.1423, -0.1849,  ...,  1.2635, -0.5557,  0.8256],
        [ 0.4915,  0.4971, -0.8622,  ..., -1.3823, -0.2737,  0.5861],
        [-0.6029, -0.0737, -0.5012,  ...,  1.6717,  1.7567,  0.9434]],
       requires_grad=True)

In [15]:
emb.weight.shape

torch.Size([13078, 64])

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [17]:
class Nextword(nn.Module):
    def __init__(self, block_size, vocab_size, emb_dim=64, hidden_size=1024):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.fc1 = nn.Linear(block_size * emb_dim, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        # x: (batch, block_size)
        x = self.emb(x)                           # (batch, block_size, emb_dim)
        x = x.view(x.shape[0], -1)                # flatten
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x)                           # logits
        return x



In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(
    X, Y, test_size=0.2, random_state=42
)
train_data = torch.utils.data.TensorDataset(X_train, Y_train)
val_data = torch.utils.data.TensorDataset(X_val, Y_val)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=32)



In [ ]:
import time
def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def print_model_summary(model):
    print("Model Summary:")
    for name, module in model.named_modules():
        if len(list(module.children())) == 0:
            params = sum(p.numel() for p in module.parameters())
            print(f"{name:<20} {params:,} parameters")
    print(f"\nTotal trainable parameters: {count_params(model):,}\n")


def train_model(model, X, Y, X_val,Y_val, epochs=500, batch_size=1024, lr=1e-3, wd=1e-4, print_every=100):
    """Train the MLP model using AdamW optimizer."""
    model.train()
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    loss_fn = nn.CrossEntropyLoss()

    losses, times = [], [] 
    for epoch in range(epochs):
        start = time.time()
        total_loss, n_batches = 0, 0
        val_loss = []

        for i in range(0, X.shape[0], batch_size):
            x_batch = X[i:i+batch_size]
            y_batch = Y[i:i+batch_size]

            logits = model(x_batch)
            loss = loss_fn(logits, y_batch)

            opt.zero_grad()
            loss.backward()
            opt.step()

            total_loss += loss.item()
            n_batches += 1

        avg_loss = total_loss / n_batches
        losses.append(avg_loss)
        times.append(time.time() - start)
        model.eval()
        with torch.no_grad():
            for i in range(0, X_val.shape[0], batch_size):
                x_batch = X_val[i:i+batch_size]
                y_batch = Y_val[i:i+batch_size]

                logits = model(x_batch)
                loss = loss_fn(logits, y_batch)
                val_loss.append(loss.item())
        
        model.train()

        if epoch % print_every == 0:
            print(f"Epoch {epoch:4d} | Loss: {avg_loss:.4f} | Time: {times[-1]:.2f}s")

    return losses, times, val_loss


In [19]:
# Create enhanced model and show before training
block_size = 4
model = Nextword(block_size, len(stoi), emb_dim, hidden_size=64).to(device)
model = torch.compile(model)

## Model parameters:
print_model_summary(model)


Model Summary:
_orig_mod.emb        836,992 parameters
_orig_mod.fc1        16,448 parameters
_orig_mod.fc2        4,160 parameters
_orig_mod.fc3        850,070 parameters

Total trainable parameters: 1,707,670



In [20]:
import torch

def generate_text(model, stoi, itos, block_size, device, start_context=None, max_len=20):
    """
    Generate a sequence of words from a trained model.

    Args:
        model: Trained PyTorch language model
        stoi: dict, mapping from word → index
        itos: dict, mapping from index → word
        block_size: int, context size expected by the model
        device: torch device ('cuda' or 'cpu')
        start_context: list of str (optional), seed words
        max_len: int, number of words to generate

    Returns:
        str: Generated text sequence
    """

    model.eval()  # evaluation mode (no dropout, etc.)

    # --- Initialize context ---
    if start_context is None:
        context = [stoi['<PAD>']] * block_size  # start with padding
    else:
        # convert words → indices, pad or truncate to block_size
        context = [stoi.get(w, 0) for w in start_context][-block_size:]
        context = [stoi['<PAD>']] * (block_size - len(context)) + context

    generated_words = []

    # --- Generate words one by one ---
    with torch.no_grad():
        for _ in range(max_len):
            x = torch.tensor(context).view(1, -1).to(device)
            y_pred = model(x)  # logits for next word
            ix = torch.distributions.categorical.Categorical(logits=y_pred).sample().item()
            word = itos[ix]

            # stop at end token 
            # if word == eos:
            #     break

            generated_words.append(word)

            # slide the context window forward
            context = context[1:] + [ix]

    model.train()  # restore training mode

    return ' '.join(generated_words)


In [21]:
text = generate_text(model, stoi, itos, block_size=4, device=device)
print(text)

W1101 22:00:39.537000 72079 torch/_inductor/utils.py:1436] [0/0] Not enough SMs to use max_autotune_gemm mode


chagrin knot crouched capless obviously diable disordered kerchief neednt reported recommend louise tracks imitated newcomers gentry assurance them manfully persuading


In [ ]:
train_model(model, X, Y, epochs=1000, batch_size=1024, lr=1e-3, wd=1e-4, print_every=100)

Epoch    0 | Loss: 6.6424 | Time: 5.94s
Epoch  100 | Loss: 2.9547 | Time: 5.50s
Epoch  200 | Loss: 2.5678 | Time: 7.35s
Epoch  300 | Loss: 2.4388 | Time: 5.91s


([6.642447591206384,
  5.82734854561942,
  5.564368361518497,
  5.379745910281227,
  5.249661996629503,
  5.147049823639885,
  5.0606246766589935,
  4.985200165945386,
  4.917814554486956,
  4.8564642482333715,
  4.799725974552215,
  4.746624820951432,
  4.6965119528392005,
  4.648916236937992,
  4.6034776369730634,
  4.559912409101214,
  4.517994042048379,
  4.477529594633315,
  4.438366445662483,
  4.400395340389675,
  4.363527203363086,
  4.32768684871613,
  4.292812990006946,
  4.25885075463189,
  4.225736962424384,
  4.193421303279816,
  4.161861426489693,
  4.131025153114682,
  4.100878952419947,
  4.071393013757373,
  4.042541642416091,
  4.014306038145035,
  3.986669013613746,
  3.9596009034959097,
  3.9330904181041415,
  3.907117441722325,
  3.881663367104909,
  3.8567225668165417,
  3.832270043993753,
  3.808303919292632,
  3.7847938552735343,
  3.761736906142462,
  3.7391223067329045,
  3.716943230704656,
  3.695186324346633,
  3.673851240248907,
  3.6529243620615155,
  3.63

In [51]:
text = generate_text(model, stoi, itos, block_size=4, device=device, start_context="maid of honor and favorite", max_len=50)
print(text)

when they spoke after his intention while he paid sleeves spirits . have you fallen there and read it was a brilliant year that review was standing . <PAD> <PAD> <PAD> <PAD> one is not great or do on my brother . the more constant individual ordered girls yourself is


In [ ]:
torch.save(model.state_dict(), 'nextword-model-lightning.pth')